In [1]:
import xml.etree.ElementTree as et

In [2]:
filename = '../enwiki-20200801-pages-articles-multistream1.xml-p1p30303'

In [3]:
for event,element in et.iterparse(filename,events=('start','end')):
    print(element.tag[1])

NameError: name 'self' is not defined